In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Federal Tax Data

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
}

In [5]:
r = requests.get(
    "https://worldpopulationreview.com/state-rankings/federal-aid-by-state",
    headers=headers,
)

In [6]:
soup = BeautifulSoup(r.text, "html.parser")
tbody = soup.find("tbody")
state_dict = {}
for tr in tbody.find_all("tr"):
    tds = tr.find_all("td")
    state = tds[1].text
    federal_dollars_spent = float(tds[3].text.replace("$", ""))
    state_dict[state] = federal_dollars_spent

In [7]:
taxes_df = pd.DataFrame(state_dict.items(), columns=["NAME", "TAXES"])

## Merge Data

In [8]:
taxes_pop_df = states_df.merge(taxes_df, on="NAME", how="right").dropna()

In [9]:
taxes_pop_df = taxes_pop_df.to_crs(9311)
taxes_pop_df.to_file("data/taxes.gpkg")